In [1]:
!pip install pandas scikit-learn matplotlib seaborn

In [ ]:

from google.colab import files
uploaded = files.upload()
import os

kaggle_file_name = list(uploaded.keys())[0]

os.makedirs('/root/.kaggle', exist_ok=True)


!mv "{kaggle_file_name}"
!chmod 600 /root/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d muhammedtausif/best-selling-mobile-phones

In [ ]:
current_files = os.listdir()
print("Current files in directory:")
print(current_files)

In [ ]:
!unzip best-selling-mobile-phones.zip

In [ ]:
import pandas as pd

df = pd.read_csv('best-selling-mobile-phones.csv')

# Display the first few rows to understand the structure
print(df.head())


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
# One-hot encode the manufacturer column
encoder = OneHotEncoder()
X = encoder.fit_transform(df[['manufacturer']]).toarray()

# Define the target (y)
y = df['units_sold_m'].values

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

#Visualize relationships
sns.scatterplot(x='manufacturer', y='units_sold_m', data=df)
plt.title('Sales vs Manufacturer')
plt.show()

#Plot actual vs predicted
plt.scatter(y_test, y_pred)
plt.xlabel('Actual Sales')
plt.ylabel('Predicted Sales')
plt.title('Actual vs Predicted Sales')
plt.show()


In [ ]:
pip install pennylane pennylane-qiskit


In [ ]:
import os
import pandas as pd
import numpy as np
import pennylane as qml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

# Load the dataset
df = pd.read_csv('best-selling-mobile-phones.csv')

# Display the first few rows to understand the structure
print(df.head())


In [ ]:
# One-hot encode the manufacturer column
encoder = OneHotEncoder()
X = encoder.fit_transform(df[['manufacturer']]).toarray() #Here I had to create an iterable array for the manufacturer so that we can map integers against each other

# Define the target (y)
y = df['units_sold_m'].values

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Normalize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
# Ensuring it's not going to overflow
num_qubits = X_train.shape[1]
dev = qml.device("default.qubit", wires=num_qubits)

In [ ]:
# Define a variational quantum circuit
def variational_circuit(weights, x):
    qml.templates.AngleEmbedding(x, wires=range(num_qubits))
    qml.templates.BasicEntanglerLayers(weights, wires=range(num_qubits))
    return [qml.expval(qml.PauliZ(i)) for i in range(num_qubits)]

#Here I did the QNode
@qml.qnode(dev, interface='tf')
def quantum_neural_network(inputs, weights):
    return variational_circuit(weights, inputs)

# Starting the QNN model
num_layers = 3
weight_shapes = {"weights": (num_layers, num_qubits)}
qlayer = qml.qnn.KerasLayer(quantum_neural_network, weight_shapes, output_dim=num_qubits)

# Classical part of the model
model = Sequential()
model.add(qlayer)
model.add(Flatten())
model.add(Dense(1))

# Compiling the model
model.compile(optimizer='adam', loss='mse')



In [ ]:

# Train the model! This is taking sm time and I don't know if it's correct
history = model.fit(X_train, y_train, epochs=20, batch_size=20, validation_data=(X_test, y_test))

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model (same as with classical)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')

# Basically just plots
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()
plt.scatter(y_test, y_pred)
plt.xlabel('Actual Sales')
plt.ylabel('Predicted Sales')
plt.title('Actual vs Predicted Sales')
plt.show()